# ML Engineer Core Workflow with MLFlow and Model Registry

This notebook provides enhanced functionality for ML Engineers to execute and manage YOLOv11 training pipelines with MLFlow experiment tracking and SageMaker Model Registry integration.

## Workflow Overview

1. **Pipeline Configuration**: Set up YOLOv11 training pipeline parameters
2. **Pipeline Execution**: Execute the training pipeline with MLFlow tracking
3. **Pipeline Monitoring**: Monitor training progress and results
4. **Model Registration**: Register trained models in SageMaker Model Registry
5. **Model Management**: Manage model versions and approval workflows

## Prerequisites

- AWS account with appropriate permissions
- AWS CLI configured with "ab" profile
- SageMaker Studio access with ML Engineer role
- Access to the drone imagery dataset in S3 bucket: `lucaskle-ab3-project-pv`
- Labeled data in YOLOv11 format
- SageMaker managed MLFlow tracking server

Let's start by importing the necessary libraries and setting up our environment.

In [ ]:
import os
import boto3
import sagemaker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import time
from IPython.display import display, HTML
import mlflow
import mlflow.sagemaker
from sagemaker.model_registry import ModelPackage
from sagemaker.model_registry.model_registry import ModelRegistry

# Set up AWS session with "ab" profile
session = boto3.Session(profile_name='ab')
sagemaker_session = sagemaker.Session(boto_session=session)
sagemaker_client = session.client('sagemaker')
region = session.region_name
account_id = session.client('sts').get_caller_identity()['Account']

# Set up MLFlow tracking
# Get SageMaker managed MLFlow tracking server URI
mlflow_tracking_uri = f"https://{region}.console.aws.amazon.com/sagemaker/home?region={region}#/mlflow"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# Set up visualization
plt.rcParams["figure.figsize"] = (12, 6)

# Define bucket name and role
BUCKET_NAME = 'lucaskle-ab3-project-pv'
ROLE_ARN = sagemaker_session.get_caller_identity_arn()

# Model Registry configuration
MODEL_PACKAGE_GROUP_NAME = "yolov11-drone-detection-models"

print(f"Data Bucket: {BUCKET_NAME}")
print(f"Region: {region}")
print(f"Account ID: {account_id}")
print(f"Role ARN: {ROLE_ARN}")
print(f"MLFlow Tracking URI: {mlflow_tracking_uri}")
print(f"Model Package Group: {MODEL_PACKAGE_GROUP_NAME}")